In [9]:
import numpy as np
import pandas as pd

import pickle

import plotly.express as px
import plotly.graph_objs as go

# if not set token, mapbox not show
mapbox_token = 'pk.eyJ1IjoidzI5dGFuZyIsImEiOiJjazZ4c3R4bGgwcWtyM2ZyeHRtejM3MmEzIn0.V-mxo1kAs_gTn7jPfTy4Yw'
px.set_mapbox_access_token(mapbox_token)

from dash import dcc
from dash import html

from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import sys
sys.path.append('../funclib')
from myclass import TensorFrame

In [11]:
gauges = pd.read_csv('data/Station_Info_ALL.csv', index_col=0)

In [12]:
with open('data/hydrometrics_tframe.pkl', 'rb') as f:
    hym_tframe_1 = pickle.load(f)
    
with open('data/hydrometrics_new_tframe.pkl', 'rb') as f:
    hym_tframe_2 = pickle.load(f)
    
hym_tframe = hym_tframe_1.append(hym_tframe_2, axis=1)

In [23]:
for hym in hym_list:
    df = hym_tframe.to_frame(hym)
    df.to_csv('data/hym/{}.csv'.format(hym))

In [33]:
a = hym_tframe.name2d
print(a)

['mean', 'median', 'std', 'skew', 'range', '5p', '10p', '25p', '75p', '90p', '95p', 'min', 'min3d', 'min7d', 'min14d', 'min30d', 'min90d', 'low_count', 'low_dur', 'max', 'max3d', 'max7d', 'max14d', 'max30d', 'max90d', 'high_count', 'high_dur', 'rising', 'falling', 'change', 'change_3d', 'n_revs', 'n_rlimb', 'n_flimb', 'rlimb_slp', 'flimb_slp', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'jan_p', 'feb_p', 'mar_p', 'apr_p', 'may_p', 'jun_p', 'jul_p', 'aug_p', 'sep_p', 'oct_p', 'nov_p', 'dec_p', 'cen_jd', '25p_jd', '50p_jd', '75p_jd', 'max_jd', 'min_jd', 'max3d_jd', 'min3d_jd', 'max7d_jd', 'min7d_jd', 'max14d_jd', 'min14d_jd', 'si', 'aut', 'aut_p', 'spr', 'spr_p', 'smr', 'smr_p', 'smr2', 'smr2_p', 'win', 'win_p', 'win2', 'win2_p', 'spr_max14d_jd', 'spr_onset_jd']


In [70]:
hovertext_frame = "{}\nName: {}\nArea: {:.1f}"

text_list = [
    hovertext_frame.format(idx, val['STATION NAME'], val['DRAINAGE AREA'])
    for idx, val in gauges.iterrows()
]

In [76]:
hovertext_frame = '<b>{}</b><br>Name: {}<br>Area: {:.1f} sqkm'

In [79]:
hovertext_frame.split('<br>')[0].replace('<b>', '').replace('</b>', '')

'{}'

In [72]:
text_list = [
    hovertext_frame.format(
        idx, val['STATION NAME'], val['DRAINAGE AREA'])
    for idx, val in gauges.iterrows()
]

In [69]:
hovertext_frame = """
    {}
    Name: {}
    Drainage Area: {:.1f}
"""

In [43]:
df_hym.loc['AB-05AA008'].index.values.astype(int)

array([1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [35]:
sel_hym = 'mean'

df_hym = pd.read_csv(
    'data/hym/{}.csv'.format(sel_hym), index_col=0)

df_mkout = pd.read_csv(
    'data/{}/{}.csv'.format(method, sel_hym), index_col=0)

In [24]:
with open('data/hym_mkout_tframe_trendfree.pkl', 'rb') as f:
    mk_tframe = pickle.load(f)

In [34]:
method_list = ['prewhiten', 'original']

for method in method_list:
    folder = 'data/{}'.format(method)
    os.mkdir(folder)
    
    with open('data/hym_mkout_tframe_{}.pkl'.format(method), 'rb') as f:
        mk_tframe = pickle.load(f)
        
    hym_list = mk_tframe.name2d
    for hym in hym_list:
        df = mk_tframe.to_frame(hym)
        df.to_csv('data/{}/{}.csv'.format(method, hym))

In [5]:
hym_name = pd.read_csv('data/Hydrometric_Name_Table.csv', index_col=0)
hym_name = hym_name.loc[hym_tframe.name2d]

In [13]:
def add_trace(fig, df_sel, size, rgba):
    fig.add_trace(
        go.Scattermapbox(
            lat=df_sel['LATITUDE'],
            lon=df_sel['LONGITUDE'],
            mode='markers',
            marker={'size': size, 'color': rgba},
            hovertext=df_sel['STATION NAME'],
            hoverinfo='text'
        )
    )

In [14]:
import plotly.graph_objects as go
import pandas as pd

df = mk_tframe.to_frame('mean').join(gauges)

fig = go.Figure()

thr = 0.05

df_sel = df[(df['pvalue'] < thr) & (df['slp'] > 0)]
add_trace(fig, df_sel, 10, 'rgba(255,0,0,.9)')

df_sel = df[(df['pvalue'] > thr) & (df['slp'] > 0)]
add_trace(fig, df_sel, 8, 'rgba(255,0,0,.2)')

df_sel = df[(df['pvalue'] < thr) & (df['slp'] < 0)]
add_trace(fig, df_sel, 10, 'rgba(0,0,255,.8)')

df_sel = df[(df['pvalue'] > thr) & (df['slp'] < 0)]
add_trace(fig, df_sel, 8, 'rgba(0,0,255,.2)')


fig.update_layout(
    title='Trend Map of Hydrometric',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox={
        'accesstoken': mapbox_token,
        'bearing': 0,
        'center': {'lat': 45, 'lon': -120},
        'zoom': 2.2,
        'style': 'light'
    },
    width=600, height=800,
    margin={'l':0, 'r':0, 't':0, 'b':0},
)

fig.show()

In [17]:
sel_hym = 'mean'
sel_sid = 'AB-05AA008'

hym_tframe.to_series(sel_hym, sel_sid)

1979    3.721562
1980    3.784877
1981    5.612082
1982    3.720411
1983    3.235890
1984    3.033397
1985    3.414274
1986    5.186411
1987    3.489452
1988    2.766055
1989    4.401479
1990    5.757699
1991    6.811562
1992    3.203233
1993    5.479836
1994    3.763342
1995    5.680164
1996    6.137041
1997    5.004219
1998    4.418904
1999    4.496055
2000    3.425644
2001    2.200932
2002    6.350247
2003    3.921096
2004    3.250027
2005    6.860329
2006    5.015562
2007    5.209123
2008    4.159753
2009    3.176685
2010    4.356110
2011    5.905260
2012    6.091397
2013    5.907945
2014    6.539315
2015    4.568767
2016    3.629041
2017    5.345644
2018    4.178247
dtype: float64

In [195]:
mk_tframe.to_series(sel_sid, sel_hym)

slp          0.028535
intp       -52.616048
pvalue       0.133611
pvalue_d     0.000000
n           40.000000
chg          1.113000
chg_r        0.243000
init         3.854000
last         4.967000
dtype: float64

In [196]:
slp, intp, pvalue, pvalue_d, n, chg, chg_r, init, last = mk_tframe.to_series(sel_sid, sel_hym)

In [25]:
trend_summary_frame = """
    |Trend Analysis||
    | ----------- | ----------- |
    |MK-Test P-Value: {:.3f}| Sen's Slpe: {:.3f}|
    |Net Change: {:.3f} | Change Rate: {:.3f}|
    |Init:  {:.3f} | Last:  {:.3f} |
    |\\# Valid Year: {:.0f}||
"""

In [26]:
mkout = mk_tframe.to_series(sel_sid, sel_hym)
slp = mkout['slp']
intp = mkout['intp']
pvalue = mkout['pvalue']

In [27]:
trend_summary = trend_summary_frame.format(
    pvalue, slp,
    mkout['chg'], mkout['chg_r'],
    mkout['init'], mkout['last'],
    mkout['n'])

In [28]:
print(trend_summary)


    |Trend Analysis||
    | ----------- | ----------- |
    |MK-Test P-Value: 0.134| Sen's Slpe: 0.029|
    |Net Change: 1.113 | Change Rate: 0.243|
    |Init:  3.854 | Last:  4.967 |
    |\# Valid Year: 40||

